In [ ]:
import pandas as pd
import numpy as np
import pickle
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import random
from sklearn.preprocessing import normalize
import csv
import os
import statistics

In [ ]:
words = pickle.load(open('final_words.pkl','rb'))

In [ ]:
#words

In [ ]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        return True 
    else: 
        return False

In [ ]:
def controlPoint(x,y):
  z = []
  for i in range(0,len(y)):
     for item in y[i].split():
         z.append(item)
  boolian = [str(common_member([str(word)], z)) for word in x]
  if common_member(['False'], boolian) == True:
    return False
  else:
    return True

In [ ]:
def compute_cosineSimilarities(lists, corpor):
  corpor = list(corpor['texts'])
  holder = []
  item = 0
  while item < len(corpor):
    if common_member(corpor[item].split(), lists) == True:
      holder.append(corpor[item])
    item = item + 1
  if len(holder) > 500:
    temp = random.sample(holder, k = 500)
    while controlPoint(lists, temp) == False:
      temp = random.sample(holder, k = 500)
    holder = temp
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform(holder)
  feature_names = vectorizer.get_feature_names()
  denselist = (vectors.todense()).tolist()
  feature_dataframe = pd.DataFrame(denselist, columns=feature_names)
  feature_gen = feature_dataframe[lists]
  feature_gen_mat = feature_gen.to_numpy()
  l2norm = np.sqrt((feature_gen_mat.T * feature_gen_mat.T).sum(axis=1))
  l2norm = l2norm.reshape(len(l2norm),1)
  norm_feature_gen_mat = (feature_gen_mat.T/l2norm).T
  V = norm_feature_gen_mat.T
  S = np.matmul(V,V.T)
  cosine_matrix_gen = pd.DataFrame(S, index = lists, columns = lists)
  return statistics.mean(list(cosine_matrix_gen.iloc[-1, 1:len(cosine_matrix_gen)]))

In [ ]:
documents = pickle.load(open('cleaned_9_14.pkl','rb'))
samples = pickle.load(open('samples_final.pkl','rb'))

In [ ]:
from google.colab import files

In [ ]:
sampler = []
for i in range(8,9):#, len(samples)):
  for j in range(0,len(samples[0])):
      sampler.append(compute_cosineSimilarities(samples[i][j], documents))
  #sampler.append(compute_cosineSimilarities(samples[i], documents))
  pickle.dump(sampler, open('{}_avg_samp_28.pkl'.format(samples[i][1][-1]),'wb'))
  #files.download('{}_avg_samp_36.pkl'.format(samples[i][1][-1]))
  sampler = []

In [ ]:
#print(statistics.mean(pickle.load(open('covid_avg_samp_26.pkl','rb'))))